# Imports

In [1]:
from pyspark.sql import SQLContext
from pyspark.sql.types import *
import folium
from ipywidgets import widgets
from IPython.display import display, clear_output
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.widgets import GraphWidget
import plotly
plotly.tools.set_credentials_file(username='silkeh', api_key='GwBAmaelgKfgkrmuL691')

separator = '\t'
#Eva
#filepath = 'D:/Workspace/_DataMining/DataSets'
#Silke
filepath = 'C:/Users/Silke/Documents/GitHub/DataMiningAP'

#output filepath for new csv files
#Eva
#outputFilepath = 'D:/Workspace/_DataMining/DataSets/IMDB_new'
#Silke
outputFilepath = 'C:/Users/Silke/Documents/GitHub/DataMiningAP/IMDB_new'

<IPython.core.display.Javascript object>

# IMDB
## Creation of RDD's and DF's

In [2]:
#TITLE.BASICS
#tconst, titleType, primaryTitle, originalTitle, isAdult, startYear, endYear, runtimeMinutes, genres
titleBasics_data = sc.textFile(filepath + '/IMDB/titleBasics.tsv')
titleBasics_rdd = titleBasics_data.filter(lambda l: 'primaryTitle' not in l).map(lambda l: l.split(separator))

#create schema for df
tbFields = []
tbFields.append(StructField('tconst', StringType(), True))
tbFields.append(StructField('titleType', StringType(), True))
tbFields.append(StructField('primaryTitle', StringType(), True))
tbFields.append(StructField('originalTitle', StringType(), True))
tbFields.append(StructField('isAdult', StringType(), True))
tbFields.append(StructField('startYear', StringType(), True))
tbFields.append(StructField('endYear', StringType(), True))
tbFields.append(StructField('runtimeMinutes', StringType(), True))
tbFields.append(StructField('genres', StringType(), True))                           

tbSchema = StructType(tbFields)

#create df
titleBasics_df = sqlContext.createDataFrame(titleBasics_rdd, tbSchema)

#clean data
titleBasics_df = titleBasics_df.drop('originalTitle', 'runtimeMinutes').sort('tconst', ascending=True)
titleBasics_df.show(5, truncate=True)

+---------+---------+--------------------+-------+---------+-------+--------------------+
|   tconst|titleType|        primaryTitle|isAdult|startYear|endYear|              genres|
+---------+---------+--------------------+-------+---------+-------+--------------------+
|tt0000001|    short|          Carmencita|      0|     1894|     \N|   Documentary,Short|
|tt0000002|    short|Le clown et ses c...|      0|     1892|     \N|     Animation,Short|
|tt0000003|    short|      Pauvre Pierrot|      0|     1892|     \N|Animation,Comedy,...|
|tt0000004|    short|         Un bon bock|      0|     1892|     \N|     Animation,Short|
|tt0000005|    short|    Blacksmith Scene|      0|     1893|     \N|        Comedy,Short|
+---------+---------+--------------------+-------+---------+-------+--------------------+
only showing top 5 rows



In [3]:
#TITLE.AKAS
#tconst, ordering, title, region, language, types, attributes, isOriginalTitle
titleAkas_data = sc.textFile(filepath + '/IMDB/titleAkas.tsv')
titleAkas_rdd = titleAkas_data.filter(lambda l: 'ordering' not in l).map(lambda l: l.split(separator))

#create schema for df
taFields = []
taFields.append(StructField('titleId', StringType(), True))
taFields.append(StructField('ordering', StringType(), True))
taFields.append(StructField('title', StringType(), True))
taFields.append(StructField('region', StringType(), True))
taFields.append(StructField('language', StringType(), True))
taFields.append(StructField('types', StringType(), True))
taFields.append(StructField('attributes', StringType(), True))
taFields.append(StructField('isOriginalTitle', StringType(), True))                       

taSchema = StructType(taFields)

#create df
titleAkas_df= sqlContext.createDataFrame(titleAkas_rdd, taSchema)

#clean data
titleAkas_df = titleAkas_df.filter(titleAkas_df.region =='US')
titleAkas_df = titleAkas_df.dropDuplicates(['titleId']).sort('titleId', ascending=True)

titleAkas_df.show(5, truncate=True)

+---------+--------+--------------------+------+--------+-----------+--------------------+---------------+
|  titleId|ordering|               title|region|language|      types|          attributes|isOriginalTitle|
+---------+--------+--------------------+------+--------+-----------+--------------------+---------------+
|tt0000001|       3|          Carmencita|    US|      \N|         \N|                  \N|              0|
|tt0000002|       5|The Clown and His...|    US|      \N|         \N|literal English t...|              0|
|tt0000005|       1| Blacksmithing Scene|    US|      \N|alternative|                  \N|              0|
|tt0000006|       3|   Chinese Opium Den|    US|      \N|         \N|                  \N|              0|
|tt0000007|       1|Corbett and Court...|    US|      \N|         \N|                  \N|              0|
+---------+--------+--------------------+------+--------+-----------+--------------------+---------------+
only showing top 5 rows



In [9]:
#TITLE.PRINCIPALS
#tconst, ordering, nconst, category, job, characters
titlePrincipals_data = sc.textFile(filepath + '/IMDB/titlePrincipals.tsv')
titlePrincipals_rdd = titlePrincipals_data.filter(lambda l: 'ordering' not in l).map(lambda l: l.split(separator))

#create schema for df
tpFields = []
tpFields.append(StructField('tconst', StringType(), True))
tpFields.append(StructField('ordering', StringType(), True))
tpFields.append(StructField('nconst', StringType(), True))
tpFields.append(StructField('category', StringType(), True))
tpFields.append(StructField('job', StringType(), True))
tpFields.append(StructField('characters', StringType(), True))                     

tpSchema = StructType(tpFields)

#create df
titlePrincipals_df = sqlContext.createDataFrame(titlePrincipals_rdd, tpSchema)

#clean data
titlePrincipals_df = titlePrincipals_df.filter(titlePrincipals_df.category.contains("actor") | titlePrincipals_df.category.contains("actress"))
titlePrincipals_df = titlePrincipals_df.filter(titlePrincipals_df.characters !="\\N")
titlePrincipals_df = titlePrincipals_df.drop('job').sort('tconst', ascending=True)

titlePrincipals_df.show(2, truncate = True)

#write to csv file
titlePrincipals_df.repartition(1).write.option("sep", "|").format('com.databricks.spark.csv').save(outputFilepath + "/titlePrincipals_df.csv",header = 'true')

+---------+--------+---------+--------+--------------+
|   tconst|ordering|   nconst|category|    characters|
+---------+--------+---------+--------+--------------+
|tt0000005|       2|nm0653042|   actor| ["Assistant"]|
|tt0000005|       1|nm0443482|   actor|["Blacksmith"]|
+---------+--------+---------+--------+--------------+
only showing top 2 rows



In [4]:
#TITLE.RATINGS
#tconst, averageRating, numVotes
titleRatings_data = sc.textFile(filepath + '/IMDB/titleRatings.tsv')
titleRatings_rdd = titleRatings_data.filter(lambda l: 'averageRating' not in l).map(lambda l: l.split(separator))

#create schema for df
trFields = []
trFields.append(StructField('tconst', StringType(), True))
trFields.append(StructField('averageRating', StringType(), True))
trFields.append(StructField('numVotes', StringType(), True))                    

trSchema = StructType(trFields)

#create df
titleRatings_df= sqlContext.createDataFrame(titleRatings_rdd, trSchema)

titleRatings_df.show(2, truncate = True)



+---------+-------------+--------+
|   tconst|averageRating|numVotes|
+---------+-------------+--------+
|tt0000001|          5.8|    1416|
|tt0000002|          6.4|     167|
+---------+-------------+--------+
only showing top 2 rows



## Join title.Basics and title.Akas and title.Ratings

In [10]:
#Aliases
Basics = titleBasics_df.alias('Basics')
Akas = titleAkas_df.alias('Akas')
Ratings = titleRatings_df.alias('Ratings')

#join title.Basics to title.Akas
AkasBasics = Akas.join(Basics, Akas.titleId == Basics.tconst, how='left')

#clean data 1st join
AkasBasics = AkasBasics.filter(AkasBasics.isAdult =='0')
AkasBasics = AkasBasics.drop('ordering', 'language', 'types', 
                               'attributes', 'isOriginalTitle', 'tconst', 
                               'primaryTitle', 'isAdult')
AkasBasics = AkasBasics.filter(AkasBasics.titleType.contains("movie"))
AkasBasics = AkasBasics.filter(AkasBasics.startYear >='1950')

#join AkasBasics with title.Ratings
titleABR = AkasBasics.join(Ratings, AkasBasics.titleId == Ratings.tconst, how='left')

#clean data 2nd join
titleABR = titleABR.drop('tconst')

titleABR.show(3, truncate = True)

#write to csv file
titleABR.repartition(1).write.option("sep", "|").format('com.databricks.spark.csv').save(outputFilepath + "/titleABR.csv", header = 'true')

+---------+-----------------+------+---------+---------+-------+--------------------+-------------+--------+
|  titleId|            title|region|titleType|startYear|endYear|              genres|averageRating|numVotes|
+---------+-----------------+------+---------+---------+-------+--------------------+-------------+--------+
|tt0042899|   Ghost Mountain|    US|    movie|     1950|     \N|Action,Adventure,...|          6.8|     791|
|tt0042917|Salt Lake Raiders|    US|    movie|     1950|     \N|             Western|          6.5|      59|
|tt0043729|  Lawless Cowboys|    US|    movie|     1951|     \N|             Western|          6.1|       9|
+---------+-----------------+------+---------+---------+-------+--------------------+-------------+--------+
only showing top 3 rows



# BabyNames
## RDD Creation
### US

In [18]:
#US Baby Names
#NationalNames.csv
#Id,Name,Year,Gender,Count
nationalname_rdd_init = (sc.textFile(filepath + '/us-baby-names/NationalNames.csv'))
nationalname_rdd = nationalname_rdd_init.filter(lambda x: 'Gender' not in x)
nationalname_rdd = nationalname_rdd.map(lambda x: x.split(","))

### Per State

In [19]:
#State Baby Names
#StateNames.csv
#Id,Name,Year,Gender,State,Count
full_statename_rdd_init = (sc.textFile(filepath + '/us-baby-names/StateNames.csv'))
full_statename_rdd = full_statename_rdd_init.filter(lambda x: 'Gender' not in x).map(lambda x: x.split(","))

## US States

In [20]:
us_states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}
us_states_abbreviations = list(us_states.keys())

# Analysis
## How many times does name x occur per year

In [21]:
name = "Arwen"

def name_per_year_plot(name):
    print(name)
    pername_nationalname_rdd = nationalname_rdd.filter(lambda x: x[1] == str(name))
    # Create a trace

    trace = go.Scatter(
        x = pername_nationalname_rdd.map(lambda x: int(x[2])).collect(),
        y = pername_nationalname_rdd.map(lambda x: int(x[4])).collect()
    )

    data = [trace]

    return(py.iplot(data, filename='basic-line'))

name_per_year_plot(name)


Arwen


## Top 10 movies for year x

In [18]:
separator = '|'

#create rdd
titleABR_data = sc.textFile(filepath + '/IMDB_new/titleABR.csv/titleABR.csv')
titleABR_rdd = titleABR_data.filter(lambda l: 'region' not in l).map(lambda l: l.split(separator))
print(titleABR_rdd.take(10))

#get top 10 movies by year
top_movies_rdd = titleABR_rdd.filter(lambda x: x[4] == '2001').filter(lambda x: x[7] != '').filter(lambda x: ('Documentary') not in x[6]).filter(lambda x: float(x[8]) >99).sortBy(lambda x: float(x[7]), False)
print(top_movies_rdd.take(10))
#top_movies_list = top_movies_rdd.map(lambda x: x[0]).top(10)
#print(top_movies_list)

[['tt0042899', 'Ghost Mountain', 'US', 'movie', '1950', '\\N', 'Action,Adventure,Western', '6.8', '791'], ['tt0042917', 'Salt Lake Raiders', 'US', 'movie', '1950', '\\N', 'Western', '6.5', '59'], ['tt0043729', 'Lawless Cowboys', 'US', 'movie', '1951', '\\N', 'Western', '6.1', '9'], ['tt0043973', 'Roadblock', 'US', 'movie', '1951', '\\N', 'Crime,Drama,Film-Noir', '6.7', '869'], ['tt0044008', 'A Christmas Carol', 'US', 'movie', '1951', '\\N', 'Drama,Fantasy', '8.1', '17122'], ['tt0044083', 'The Strip', 'US', 'movie', '1951', '\\N', 'Crime,Drama,Film-Noir', '6.1', '509'], ['tt0044431', "Damon Runyon's Bloodhounds of Broadway", 'US', 'movie', '1952', '\\N', 'Comedy,Musical', '6.2', '193'], ['tt0044768', 'Desperate Decision', 'US', 'movie', '1952', '\\N', 'Drama', '6.5', '19'], ['tt0044770', 'Yolanda', 'US', 'movie', '1953', '\\N', 'Adventure', '6.8', '38'], ['tt0045015', 'Bombay Waterfront', 'US', 'movie', '1952', '\\N', 'Mystery', '5.9', '77']]
[['tt0120737', 'The Lord of the Rings: The F